# Executive Summary
**This notebook demonstrates an AI-powered multi-agent system that analyzes workout pain patterns, diagnoses probable causes, and generates evidence-based recovery plans. It combines modular agent reasoning, LLM-based inference, and a lightweight Flask interface for interactive analysis.**






In [97]:
from IPython.display import SVG, display
display(SVG('/content/drive/MyDrive/WorkoutFormChecker/agentic_flow_diagram.svg'))

# Connect Google driver

In [78]:
!mkdir -p WorkoutFormChecker/knowledge_base
%cd WorkoutFormChecker/
from google.colab import drive
drive.mount('/content/drive')
!cp -r /content/WorkoutFormChecker /content/drive/MyDrive/
%cd /content/drive/MyDrive/WorkoutFormChecker


/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/WorkoutFormChecker
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/WorkoutFormChecker


In [5]:
%cd /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
!grep -n "run" master.py

/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
110:def run(user_input: str):
129:    return run(user_input)
149:    result = run(test_input)  # Uses module-level run() function


#Installation

In [ ]:
%cd /content/drive/MyDrive/WorkoutFormChecker/
!pip install -r requirements.txt

/content/drive/MyDrive/WorkoutFormChecker
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 174.6/174.6 kB 3.5 MB/s eta 0:00:00
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 86.0/86.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 123.5/123.5 kB 8.6 MB/s eta 0:00:00
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ‚îÅ 61.0/61.0 kB 5.0 M

# Verification of Installation

In [ ]:
import fastapi
import chromadb
import transformers
import torch

print("fastapi version:", fastapi.__version__)
print("chromadb version:", chromadb.__version__)
print("transformers version:", transformers.__version__)
print("torch version:", torch.__version__)


fastapi version: 0.104.1
chromadb version: 1.2.1
transformers version: 4.35.2
torch version: 2.3.1+cu121


/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# GPU or CPU?

In [ ]:
import torch

print(" Checking GPU")
print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
print(f"CUDA version: {torch.version.cuda}")
print(f"GPU Name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")
print(f"GPU  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f}GB" if torch.cuda.is_available() else 'No GPU')

 Checking GPU
Is CUDA supported by this system? False
CUDA version: 12.1
GPU Name: No GPU
No GPU


# Install Dependencies

In [ ]:
!pip install -r requirements.txt

In [ ]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv('/content/drive/MyDrive/WorkoutFormChecker/.env')

# Now you can access the variables
NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
RETRIEVER_KEY = os.getenv("RETRIEVER_KEY")

client = OpenAI(
  base_url="https://integrate.api.nvidia.com/v1",
  api_key=NVIDIA_API_KEY
)

completion = client.chat.completions.create(
  model="nvidia/llama-3.1-nemotron-nano-8b-v1",
  messages=[{"role":"user","content":"What causes Knee pain during squats"}],
  temperature=0,
  top_p=0.95,
  max_tokens=4096,
  frequency_penalty=0,
  presence_penalty=0,
  stream=True
)

for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")




<think>

</think>

Knee pain during squats can arise from several factors, and it's essential to identify the underlying cause for effective management. Here are common causes and potential solutions:

### **Common Causes of Knee Pain During Squats:**

1. **Knee Injury or Strain**:
   - **Meniscus Tear**: A torn meniscus (cartilage in the knee joint) can cause pain, especially during activities like squats. Symptoms include popping sensation, swelling, and instability.
   - **Ligament Tear**: Tears in ligaments (e.g., ACL, PCL) can lead to pain and instability. ACL tears are more common in women and often cause pain during squatting.
   - **Knee Cap (Meniscus) Dislocation**: A dislocated meniscus can cause pain and instability, especially when the knee is twisted or bent.

2. **Overuse or Repetitive Stress**:
   - **Knee Overuse**: Squatting with heavy weights or high volume can overuse the knee joint, leading to pain. Proper form (keeping knees aligned, not bending too deeply) can hel

In [ ]:
from openai import OpenAI

client = OpenAI(
  api_key=RETRIEVER_KEY,
  base_url="https://integrate.api.nvidia.com/v1"
)

response = client.embeddings.create(
    input=["What causes Knee pain during squats"],
    model="nvidia/nv-embedqa-e5-v5",
    encoding_format="float",
    extra_body={"input_type": "query", "truncate": "NONE"}
)

print(response.data[0].embedding)


[0.01580810546875, 0.008697509765625, -0.00390625, -0.0274658203125, 0.07696533203125, 0.045684814453125, 0.00759124755859375, -0.060394287109375, 0.015045166015625, -0.059112548828125, -0.009307861328125, -0.0008234977722167969, -0.038482666015625, -0.030975341796875, 0.02789306640625, -0.01458740234375, 0.00812530517578125, -0.037750244140625, 0.0262603759765625, -0.03656005859375, 0.00719451904296875, 0.047454833984375, 0.003345489501953125, 0.04150390625, -0.034454345703125, -0.031707763671875, 0.04644775390625, 0.03887939453125, -0.0025920867919921875, 0.0291290283203125, -0.042999267578125, -0.005077362060546875, -0.044586181640625, -0.04254150390625, -0.002918243408203125, -0.032867431640625, 0.00786590576171875, 0.012481689453125, -0.00848388671875, 0.037445068359375, -0.033599853515625, 0.0157318115234375, -0.0106964111328125, -0.00037860870361328125, -0.06610107421875, 0.057159423828125, 0.0219268798828125, -0.0162353515625, 0.047332763671875, -0.00922393798828125, 0.00796508

# Master Orchestrator Testing

In [95]:
%cd '/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra'
!python3 master.py

/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
BaseAgent class with log_action created!
‚úÖ WebSearchTool created (REAL search only, no fake fallback)!
üß™ Running Agentic MasterOrchestrator in standalone mode...

üéº Initializing Master Orchestrator with reasoning loop...
‚úÖ All agents initialized successfully!

ü§ñ AGENTIC MULTI-AGENT ANALYSIS STARTED

üîÑ Iteration 1/3

üìù STEP 1: Parsing user input...
‚úÖ Extracted: none | sharp shoulder | none
üß© Extended reasoning fields integrated for deeper context.

üèãÔ∏è STEP 2: Analyzing form patterns and deviations...
üß† Form Analysis Insight: 1. Most likely form breakdown: Inefficacious or limited range of motion during shoulder abduction exercises, possibly due to poor form or compensatory mechanisms. 
2. Biomechanical explanation: Pain is likely caused by soft tissue impingement or instability in the rotator cuff or posterior deltoid, exacerbated by limited abduction range. 
3. What to check in form: Ensure proper

In [91]:
from IPython.display import HTML

# Force reload by reading the file again
with open('/content/drive/MyDrive/WorkoutFormChecker/frontend.html', 'r') as f:
    html_content = f.read()

HTML(html_content)

In [ ]:
%cd /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
!python master.py

/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra
BaseAgent class with log_action created!
 Testing Master Orchestrator...

 Initializing Master Orchestrator...
 All agents initialized!

MULTI-AGENT ANALYSIS STARTED

 STEP 1: Parsing user input...
 Extracted: squats | right knee | after workout

 STEP 2: Analyzing exercise form...
 Form analysis complete (confidence: high)

 STEP 3: Diagnosing injury pattern...
 Diagnosis complete (confidence: high)

 STEP 4: Researching supporting evidence...
 Searching knowledge base for: squats right knee form correction
 Searching web for: squats right knee injury treatment 2024
 Found 1 web sources

 STEP 5: Creating personalized action plan...
 Action plan generated!

 MULTI-AGENT ANALYSIS COMPLETE

 FINAL RESULTS

ACTION PLAN:
**ROOT CAUSE**
Patellar Tracking Deviation caused by right knee misalignment, compressing lateral meniscus.

**IMMEDIATE ACTION (Next Workout)**
1. **Squat with knee alignment focus**: Assume a wider stance and eng

In [ ]:
!zip -r my_project.zip /content/drive/MyDrive/WorkoutFormChecker/

  adding: content/drive/MyDrive/WorkoutFormChecker/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/excercise_forms.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/injury_patterns.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/corrective_excercises.txt (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/knowledge_base/load_kb.ipynb (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/README.md (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.gitignore (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/requirements.txt (deflated 32%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.env (deflated 14%)
  adding: content/drive/MyDrive/WorkoutFormChecker/.ipynb_checkpoints/ (stored 0%)
  adding: content/drive/MyDrive/WorkoutFormChecker/__pycache__/ (stored 0%)
  add

In [ ]:
from google.colab import files
files.download("WorkoutFormChecker.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Check if server.py exists
!ls -la /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py

# Check if frontend.html exists (might be in different location)
!ls -la /content/drive/MyDrive/WorkoutFormChecker/frontend.html

-rw------- 1 root root 6634 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py
-rw------- 1 root root 13717 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/frontend.html


In [ ]:
!ls -la /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py

-rw------- 1 root root 6634 Oct 25 15:12 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py


##Testing of FLASK APP


In [ ]:
from threading import Thread
import time

def run_flask():
    import os
    os.chdir('/content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra')
    from server import app  # or however your app is defined
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in background
thread = Thread(target=run_flask)
thread.start()

# Give it time to start
time.sleep(3)
print("‚úÖ Flask server should be running on port 5000")

 * Serving Flask app 'server'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


‚úÖ Flask server should be running on port 5000


In [13]:
from pyngrok import ngrok
from dotenv import load_dotenv
import os

load_dotenv('/content/drive/MyDrive/WorkoutFormChecker/.env')
NGROK_TOKEN = os.getenv("NGROK_AUTH_TOKEN")
ngrok.set_auth_token(NGROK_TOKEN)



# Server Connection Issue
## -> Kill the server.
## -> Connect to 8081.
## -> Expose the Base URl.

In [92]:
!pkill -f "server.py" || true
!pkill -f "flask" || true
!pkill -f "http.server"
from pyngrok import ngrok
ngrok.kill()



^C
^C


In [93]:
!nohup python3 /content/drive/MyDrive/WorkoutFormChecker/MasterOrchestra/server.py > server.log 2>&1 &
!sleep 3
!lsof -i :5000


COMMAND    PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
python3 130153 root    5u  IPv4 3968553      0t0  TCP *:5000 (LISTEN)
python3 130161 root    5u  IPv4 3968553      0t0  TCP *:5000 (LISTEN)
python3 130161 root    6u  IPv4 3968553      0t0  TCP *:5000 (LISTEN)


In [94]:
from pyngrok import ngrok
backend_url = ngrok.connect(5000)
print("Backend URL:", backend_url.public_url)


Backend URL: https://rebeca-groutiest-incorporeally.ngrok-free.dev


### Conclusion
**This notebook demonstrates how structured reasoning and modular agent design can automate workout injury diagnostics. Future versions will integrate pose estimation and personalized recovery tracking to bridge the gap between AI insights and real-world physical therapy.**
